In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import cartopy.crs as ccrs
%matplotlib

Using matplotlib backend: Qt5Agg


In [3]:
ds = xr.open_dataset("C:/Users/dmres/OneDrive/Desktop/Interview/baja-fixed.nc")

In [4]:
ds

<xarray.Dataset>
Dimensions:  (T: 3926, X: 74, Y: 67)
Coordinates:
  * T        (T) datetime64[ns] 2010-01-01T12:00:00 ... 2020-09-30T12:00:00
  * X        (X) float32 -121.625 -121.375 -121.125 ... -103.625 -103.375
  * Y        (Y) float32 19.125 19.375 19.625 19.875 ... 35.125 35.375 35.625
Data variables:
    prcp     (T, Y, X) float32 ...

In [5]:
#play around with a single month
ds.sel(T="2010-1")
timestep1_mean = ds.sel(T="2010-1").mean(dim="T")

#convert to array so it can be quickly plotted
timestep1_array = timestep1_mean.to_array()
timestep1_array.plot()

C:\Users\dmres\Anaconda3\lib\site-packages\xarray\core\nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [123]:
#reducing the full dataset so it just includes monthly means
ds_resample = ds.resample(T="1MS").reduce(np.mean)
ds_resample

<xarray.Dataset>
Dimensions:  (T: 129, X: 74, Y: 67)
Coordinates:
  * T        (T) datetime64[ns] 2010-01-01 2010-02-01 ... 2020-08-01 2020-09-01
  * X        (X) float32 -121.625 -121.375 -121.125 ... -103.625 -103.375
  * Y        (Y) float32 19.125 19.375 19.625 19.875 ... 35.125 35.375 35.625
Data variables:
    prcp     (T, Y, X) float32 nan nan nan ... 0.7658498 0.73326963 0.79116285

In [124]:
#t = ds_resample.isel(T=slice(0,9)).to_array()
t = ds_resample.sel(T=slice('2010-01-01T12:00:00.000000000','2010-10-01T12:00:00.000000000')).to_array()

simple = t.plot(x="X", y="Y", col="T", col_wrap=3)

In [6]:
#generate averages for every month over the entire dataset
ds_clim = ds.groupby('T.month').mean('T')
ds_clim

<xarray.Dataset>
Dimensions:  (X: 74, Y: 67, month: 12)
Coordinates:
  * X        (X) float32 -121.625 -121.375 -121.125 ... -103.625 -103.375
  * Y        (Y) float32 19.125 19.375 19.625 19.875 ... 35.125 35.375 35.625
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    prcp     (month, Y, X) float32 nan nan nan ... 0.41699877 0.38865188

In [7]:
#convert to array and plot the data
ds_clim_array = ds_clim.to_array()
clim_plot = ds_clim_array.plot(x="X", y="Y", col="month", col_wrap=3)
plt.savefig('C:/Users/dmres/monthly_climatoloy.png')

In [175]:
month1 = ds_clim.sel(month=1).to_array().plot()

In [216]:
for i in ds_clim['month']:
    plt.figure()
    mnth = ds_clim.sel(month=i).to_array().plot()
    #plot = mnth.to_array().plot()



In [17]:
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter

fig = plt.figure()

ims = []
for i in ds_clim['month']:
    mnth = ds_clim.sel(month=i).to_array().plot(add_colorbar=False)
    #mnth['prcp'].plot(add_colorbar=False)
    ims.append([mnth])

    
ani = animation.ArtistAnimation(fig, ims, interval=200, blit=True, repeat_delay=100)
#ani.save("C:/Users/dmres/IRI_plots/dynamic.gif", writer=PillowWriter(fps=24))
plt.show()

In [10]:
for i in ds_clim['month']:
    plt.figure()
    mnth = ds_clim.sel(month=i)
    mnth['prcp'].plot()